## Background
The main goal for this case is for us to understand your ability to understand the data provided and draw critical insights from the dataset. There are four tables for you to work with: 
1. Customer Accounts (id, name): a list of our customers with their names redacted and a unique identifier
2. Financial Accounts (id, customer_account_id, name): a list of bank accounts belonging to our customers
    * Customer_account_id: The ID of the customer that owns this bank account from Customer Accounts (table 1)
    * Id, name: Unique financial account identifier and redacted name of the account
3. Financial Balances (id, account_id, amount, accrual_date): a reading of the beginning-of-day (BoD) balance for each financial account
    * Account_id: the ID of the financial account (table 2) that the balance belongs to
    * Amount and accrual_date: The balance (in dollars) and date this reading was taken
4. Financial Transactions (id, account_id, amount, accrual_date): list of transactions incoming and outgoing for each financial account
    * Account_id: the ID of the financial account (table 2) that the transaction belongs to
    * Amount and accrual_date: the amount (in dollars) of each transaction and the date when the transaction occurred. Negative transactions are money leaving the bank account and positive transactions are money coming in.

In [254]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option("display.max_rows", 999)

In [154]:
#load csv
customer_accounts = pd.read_csv("customer_accounts.csv")
financial_accounts = pd.read_csv("financial_accounts.csv")
financial_balances = pd.read_csv("financial_balances.csv")
financial_transactions = pd.read_csv("financial_transactions.csv")

In [155]:
#pre-processing
financial_balances['accrual_date'] = pd.to_datetime(financial_balances['accrual_date'],format = "%Y-%m-%d")
financial_transactions['accrual_date'] = pd.to_datetime(financial_transactions['accrual_date'],format = "%Y-%m-%d")

## Questions

For the following questions:
- `Customer A` = "a_53"
- `Customer A Account 1` = "a_53_1" 
- `Customer A Balance Date` = "2020-07-04
- `Customer B` = "a_32" 
- `Customer B Balance Date` = "2020-08-12"

In [100]:
# recommended to run
customer_a = 'a_53'
customer_a_account_1 = 'a_53_1'
customer_a_balance_date = '2020-07-04'
customer_b = 'a_32'
customer_b_balance_date= '2020-08-12'

In [101]:
for df in [customer_accounts,financial_accounts,financial_balances,financial_transactions]:
    display(df.head(3))

,id,name
0,a_0,a_0_company
1,a_1,a_1_company
2,a_2,a_2_company


,id,customer_account_id,name
0,a_0_0,a_0,a_0_0_fin_account
1,a_0_1,a_0,a_0_1_fin_account
2,a_0_2,a_0,a_0_2_fin_account


,id,account_id,accrual_date,amount
0,a_0_0_bal_0,a_0_0,2019-12-24,9950997.0
1,a_0_0_bal_1,a_0_0,2019-12-25,9950749.0
2,a_0_0_bal_2,a_0_0,2019-12-26,9949890.0


,account_id,accrual_date,amount,id
0,a_0_0,2019-12-24,-248.0,a_0_0_0
1,a_0_0,2019-12-25,-859.0,a_0_0_1
2,a_0_0,2019-12-26,-776.0,a_0_0_2


### Question 1.
Recommended time usage: ~10 minutes

(a) What is the average historic daily balance (beginning-of-day) for `Customer A Account 1`? Round to the nearest integer. ( 1 Point )

(b) What is `Customer A`'s global balance (sum of all beginning-of-day balances across all financial accounts associated with `Customer A`) on `Customer A Balance Date`? Round to the nearest integer. ( 1 Point )


In [102]:
#1a
answer_1a = int(np.mean(financial_balances[financial_balances['account_id'] == customer_a_account_1]['amount']))
print(answer_1a)

10008868


In [103]:
#1b
customer_a_all_accounts = financial_accounts[financial_accounts['customer_account_id'] == customer_a]['id'].unique()
answer_1b = int(np.sum(financial_balances[financial_balances['account_id'].isin(customer_a_all_accounts)]['amount']))
print(answer_1b)

5708833407


### Question 2.

Recommended time usage: ~10 minutes

`financial_balances.csv` shows you the beginning-of-day (BoD) balance, but transactions happened throughout the day. Calculate the end-of-day (EoD) global balance for `Customer A` on `Customer A Balance Date`. ( 1 point )

In [106]:
#2

#create an array of all sub accounts under customer a
customer_a_all_accounts = financial_accounts[financial_accounts['customer_account_id'] == customer_a]['id'].unique()

#beginning balance for Customer A on Customer A Balance Date
beg_bal_cust_a_balance_date = np.sum(financial_balances[(financial_balances['account_id'].isin(customer_a_all_accounts)) & (financial_balances['accrual_date'] == customer_a_balance_date)]['amount'])

#sum of transactions throughout the day for Customer A on Customer A Balance Date
sum_of_trans_cust_a_balance_date = np.sum(financial_transactions[(financial_transactions['account_id'].isin(customer_a_all_accounts)) & (financial_transactions['accrual_date'] == customer_a_balance_date)].groupby(['account_id']).agg({"amount":"sum"}).reset_index()['amount'])

answer_2 = int(np.sum([beg_bal_cust_a_balance_date,sum_of_trans_cust_a_balance_date]))
print(answer_2)

39799795


### Question 3.
Recommended time usage: ~20 minutes

(a) Among the `customer_account_id`s without any missing or null/NaN beginning-of-day balance for any date in between their first and their last `accrual_date`s, which `customer_account_id` has the largest variance of the sums of daily transactions amounts across all accounts? ( 1 points )

(b) Among the `customer_account_id`s without any missing or null/NaN beginning-of-day balance for any date in between their first and their last `accrual_date`s, which `customer_account_id` has the largest variance of the EoD (end-of-day, not beginning-of-day) global balances? ( 2 points )


In [209]:
#3a
financial_balances['min_accrual_date'] = financial_balances.groupby('account_id')['accrual_date'].transform(np.min)
financial_balances['max_accrual_date'] = financial_balances.groupby('account_id')['accrual_date'].transform(np.max)
financial_balances['n_days_between_min_max_accural_dates'] = ((financial_balances['max_accrual_date'] - financial_balances['min_accrual_date']) / np.timedelta64(1, 'D')).astype('int64')
financial_balances['n_unique_accrual_dates'] = financial_balances.groupby('account_id')['accrual_date'].transform(lambda x: len(set(x)))
financial_balances['account_missing_any_bod_min_to_max_accrual'] = np.where(financial_balances['n_days_between_min_max_accural_dates'] + 1 == financial_balances['n_unique_accrual_dates'],0,1)

#filter df to only the records of accounts with non missing bod balance data 
non_missing_account_ids = set(financial_balances[financial_balances['account_missing_any_bod_min_to_max_accrual'] == 0]['account_id'])
financial_balances_nn = financial_balances[financial_balances['account_id'].isin(non_missing_account_ids)]
financial_transactions_nn = financial_transactions[financial_transactions['account_id'].isin(non_missing_account_ids)]

answer_3a = financial_transactions_nn.groupby(['account_id','accrual_date']).agg({"amount":np.sum}).reset_index().groupby('account_id').agg({"amount":np.std}).rename(columns={"amount": "var_of_sums_of_daily_trans"}).idxmax().values[0]
print(answer_3a)

a_65_2


In [229]:
#3b

#sum of transactions throughout the day for Customer X on Customer Y Balance Date
sum_daily_trans = financial_transactions_nn.groupby(['account_id','accrual_date']).agg({"amount":np.sum}).reset_index()

financial_balances_nn_merged = financial_balances_nn.merge(sum_daily_trans,left_on=["account_id","accrual_date"],right_on=["account_id","accrual_date"])

financial_balances_nn_merged['amount_EoD'] = financial_balances_nn_merged['amount_x'] + financial_balances_nn_merged['amount_y']

answer_3b = financial_balances_nn_merged.groupby('account_id').agg({"amount_EoD":np.std}).idxmax().values[0]
print(answer_3b)

a_139_1


___________________________________________________________________________________________________

### Question 4. 

Recommended time usage: ~60+ minutes

(a) Calculate the `end-of-day` global balance for `Customer B` on `Customer B Balance Date`. Fill in this EoD balance as the answer in the submission csv. ( 4 points )

(b) Compute the `end-of-day` balances for all `financial_accounts`. Fill in as many missing dates as possible. The final dataset should include daily `end-of-day` balances for all `financial_accounts` as a csv in a similar format as `financial_balances.csv` (include just those columns: [`account_id, accrual_date, amount`]) and named `financial_balances_submission.csv`. ( 10 points )


In [261]:
financial_accounts[financial_accounts['customer_account_id'] == customer_b]

,id,customer_account_id,name
129,a_32_0,a_32,a_32_0_fin_account
130,a_32_1,a_32,a_32_1_fin_account


In [608]:
try:
    #test_list = ['a_32_0','a_32_1']
    sum_daily_trans_original = financial_transactions.groupby(['account_id','accrual_date']).agg({"amount":np.sum}).reset_index()

    frames = []

    for account in financial_balances['account_id'].unique():
    #for account in test_list:
        min_accrual_date = financial_balances[financial_balances['account_id'] == account]['accrual_date'].min()
        max_accrual_date = financial_balances[financial_balances['account_id'] == account]['accrual_date'].max()
        idx = pd.date_range(start=min_accrual_date,end=max_accrual_date)
        s = financial_balances[financial_balances['account_id'] == account].set_index("accrual_date")
        s.index = pd.DatetimeIndex(s.index)
        s = s.reindex(idx,fill_value=account)

        s_merged = s.reset_index().rename(columns={"index": "accrual_date"})[['accrual_date','account_id','amount']].merge(sum_daily_trans_original,left_on=["account_id","accrual_date"],right_on=["account_id","accrual_date"])
        s_merged['amount_x'] = [np.nan if isinstance(x, str) else x for x in s_merged['amount_x']]
        s_merged = s_merged[['accrual_date','account_id','amount_x','amount_y']]


        for ix, row in s_merged.iterrows():
            if ix < len(s_merged)-1:
                if np.isnan(s_merged.loc[ix,'amount_x']):
                    s_merged.loc[ix, 'amount_x'] = s_merged.loc[ix-1,'amount_x'] + s_merged.loc[ix-1,'amount_y']
                else:
                    s_merged.loc[ix, 'amount_x'] = s_merged.loc[ix,'amount_x']
        frames.append(s_merged)
except KeyError:
    pass

In [609]:
result = pd.concat(frames)
result['amount_eod'] = result['amount_x'] + result['amount_y']

In [610]:
result

,accrual_date,account_id,amount_x,amount_y,amount_eod
0,2019-12-24,a_0_0,9950997.0,-248.0,9950749.0
1,2019-12-25,a_0_0,9950749.0,-859.0,9949890.0
2,2019-12-26,a_0_0,9949890.0,-776.0,9949114.0
3,2019-12-27,a_0_0,9949114.0,-605.0,9948509.0
4,2019-12-28,a_0_0,9948509.0,-33.0,9948476.0
...,...,...,...,...,...
59,2020-08-28,a_65_3,10213656.0,-1507.0,10212149.0
60,2020-08-29,a_65_3,10212149.0,-963.0,10211186.0
61,2020-08-30,a_65_3,10211186.0,-1177.0,10210009.0
62,2020-08-31,a_65_3,10210009.0,-539.0,10209470.0


In [634]:
answer_4a = result[(result['account_id'] == 'a_32_0') & (result['accrual_date'] == customer_b_balance_date)]['amount_eod'].values[0] + result[(result['account_id'] == 'a_32_1') & (result['accrual_date'] == customer_b_balance_date)]['amount_eod'].values[0]
print(answer_4a)

20019370.0


In [635]:
answer_4b = result[["account_id", "accrual_date", "amount_eod"]]
answer_4b.to_csv("answer_4b.csv",index=False)